In [ ]:
import os
import tensorflow as tf
print('tf', tf.__version__)
#######################
import cv2
#######################
import lib.utils.utils as ut
import lib.utils.dataset as utds
import lib.utils.slm as utslm
import lib.oamlib as ol
import lib.CITL as CITL
import lib.modulate as mod
import lib.CITL_trainer as CITL_trainer
import lib.oamnet_trainer as oamnet_trainer

from lib.device_manager import DeviceManager

# This script showcases OAM-multiplexed holography using deep learning and Camera-in-the-loop (CITL) optimization.

In [ ]:
# Setting up OAM-multiplexed holography setup with a single SLM and a camera.
suimg = ol.Setup(N = [384,512,256], L = .2,ps = 1)      # Image setup with a 384x512 resolution
suslm = ol.Setup(N = [1024,1024,256], L = .2,ps = 1)    # SLM setup with a 1024x1024 resolution
d = 8   # Sampling interval in pixels
Nch = 12    # Number of channels (OAM modes) to be used in the multiplexing
dl = 2  # Topological charge spacing between OAM modes
image_size = [suimg.Ny, suimg.Nx] 

l_list = tf.constant(range(-Nch//2*dl, Nch//2*dl, dl), tf.float32) # OAM modes' topological charges
oam = ol.OAM(suimg, suslm, offpp = [.5,.15], l_list = l_list, d = d, slmpadf = 0, RA = 512)     # Class providing necessary functions for OAM-multiplexed holography
modulation = mod.Modulation_1SLM(oam)   # Modulation class for OAM-multiplexed holography with a single SLM


In [ ]:
# Setting up the device manager to control the SLM and camera.
device_manager = DeviceManager()    # This class manages the SLM and camera devices.
device_manager.setup_devices(exptime = 10000)   # Set the exposure time

holo_grid = cv2.imread('./grid/grid_15_gs1000'+'.png', cv2.IMREAD_UNCHANGED)[None,...,None] # Load a hologram that produces a grid pattern corresponding to the 2D sampling array.
holo_grid = tf.constant(holo_grid,tf.uint8)
gridimg = device_manager.show_capture(holo_grid, slmpause = 400) # Display the grid hologram and capture the image from the camera.
#gridimg = device_manager.show_capture_multi(holo_grid, slmpause = 400) # For future use working with two SLMs.

ut.draw(gridimg[0,1000:1160,1200:1360,0], size = 1) # Draw a sample of the captured sampling array image.

# Setting up the CITL experiment class.
experimenter = CITL.ExperimentClass(oam, device_manager, modulation, holo_grid, dh=0.1, prom=0.1, win_size= 3, slmpause= 300)



In [ ]:
# Setting up the training dataset for learning parameterized propagation model via CITL calibration.
homepath = 'C:\\Users\\nima_\\CITL_DS_pad15_f64_mix_symTC\\'
#omepath = 'C:\\Users\\nima_\\CITL_DS\\'
holopath = homepath + f'd8_L{oam.dl}_N{oam.Nch}\\'

# Creating a dataset from the holograms stored in binary files.
holo_ds = utds.binarydataset(1000,[holopath], n_items = 2, types = [tf.uint8,tf.float32], sf = False, Batch = None)

lrs = {0: 0.01, 250: 0.001, 900: 0.0001}
# Training the CITL model with the specified parameters.
propmodel = CITL_trainer.train("1SLM", oam, experimenter, holo_ds, lrs, 300, verbose = False)

In [ ]:
# Setting up the training dataset for OAM-multiplexed holography using neural networks (OAMNET).
trainpathbin = 'C:/Users/nima_/My Drive/DS/DIV2K_train_HR_bin/'
circpath = 'C:/Users/nima_/My Drive/DS/circles/'

# Creating a custom dataset for training the OAMNET model.
train_samp_ds = ut.customdataset([trainpathbin,circpath],Nch, 12, insize=[64,48,1], outsize=None, Batch = 1)

# Pretrained OAMNET model assuming an ideal propagation model.
OAMNET_basic = tf.keras.models.load_model(f'./nets/NN64f_pad15_d8_L{oam.dl}_N{oam.Nch}_mix')


OAMNET_prop = tf.keras.models.load_model(f'./nets/NN64f_pad15_d8_L{oam.dl}_N{oam.Nch}_mix')
# Training the OAMNET model assuming a parameterized propagation model.
oamnet_trainer.train(OAMNET_prop, train_samp_ds, oam, propmodel, modulation,  
                     save_dir= './nets_prop', name= 'NN_64f', verbose = False)


In [ ]:
"""
Gathering the results for OAM-multiplexed holography using various methods.
    - GD_basic: Gradient descent method assuming an ideal propagation model.
    - GD_prop: Gradient descent method assuming a parameterized propagation model.
    - NN_basic: Neural network method assuming an ideal propagation model.
    - NN_prop: Neural network method assuming a parameterized propagation model.
    - GS_basic: Conventional Phase Hologram (Conv. PH) method via the GS algorithm assuming an ideal propagation model.
    - GS_prop: Conventional Phase Hologram (Conv. PH) method via the GS algorithm  assuming a parameterized propagation model.
"""

test_ds_path = 'C:/Users/nima_/My Drive/DS/DIV2K_valid_HR/'
logo_ds_path = 'C:/Users/nima_/My Drive/OAMsamples/logo/'

I_true_logo = CITL.load_or_create_samples(12, 2, image_size, 'logo', logo_ds_path, sample_path = "./Itrue", regen = True)
I_true_natural = CITL.load_or_create_samples(12, 2, image_size, 'natural', test_ds_path, sample_path = "./Itrue", regen = True)
I_true_test = I_true_natural + I_true_logo
I_true_test = [I_true_test[5]]

resultfolder = './result'
casename = f'L{oam.dl}_N{oam.Nch}'

# Methods that solves, simulates, performs experiment measurements and saves results for OAM-multiplexed holography using various methods.
GD_prop_fun = lambda I: CITL.measure_gd(I, oam, path, experimenter, 3, propmodel, modulation, itr= 100)
GD_basic_fun = lambda I: CITL.measure_gd(I, oam, path, experimenter, 3, None, None, itr= 100)

NN_prop_fun = lambda I: CITL.measure_nn(I, oam, path, OAMNET_prop, experimenter, 3, propmodel, modulation)
NN_basic_fun = lambda I: CITL.measure_nn(I, oam, path, OAMNET_basic, experimenter, 3, None, None)

GS_prop_fun = lambda I: CITL.measure_gs(I, oam, path, experimenter, 3, propmodel, itr= 200)
GS_basic_fun = lambda I: CITL.measure_gs(I, oam, path, experimenter, 3, None, itr= 200)

methods = [{"name": "gd_prop", "fun": GD_prop_fun},
           {"name": "gd_basic", "fun": GD_basic_fun},
           {"name": "nn_prop", "fun": NN_prop_fun},
           {"name": "nn_basic", "fun": NN_basic_fun},
           {"name": "gs_prop", "fun": GS_prop_fun},
           {"name": "gs_basic", "fun": GS_basic_fun}
           ]

# Running methods to solve, simulate, perform experiment measurements and save results.
for method in methods:
    path = os.path.join(resultfolder, casename , method["name"])
    method["fun"](I_true_test)
    print(method["name"], "is done")





In [ ]:
# Releasing the devices after the experiment is done.
device_manager.release_devices()
device_manager = []